## Instructions ##
1. Map top 10% snps to genes
2. Use additive model 


In [1]:
import pandas as pd 
import numpy as np
import bisect as bs
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd
import scipy.stats as stat

In [2]:
SNP_url = "./1011geneSNP.csv"
Gene_url = "./GCF_000146045.2_R64_genomic.gff"

snp_yeast_matrix = pd.read_csv(SNP_url, sep = ',' , error_bad_lines=False)
del snp_yeast_matrix['#CHROM']
pos_list = list(snp_yeast_matrix['POS'])
del snp_yeast_matrix['POS']
del snp_yeast_matrix['REF']
del snp_yeast_matrix['ALT']
del snp_yeast_matrix['ANN[*].GENE']
del snp_yeast_matrix['ANN[*].GENEID']
index_list = snp_yeast_matrix["ID"].tolist()
del snp_yeast_matrix['ID']
snp_yeast_matrix = snp_yeast_matrix.transpose()
column_names = list(snp_yeast_matrix.index)
to_delete = ["ABC", "ABF", "ASP", "BGS","BGF", "BHL","BHQ","BID","BIR","BKG","BKN","CFH","SACE_YAL","SACE_YBA",
             "SACE_YBB","SACE_YBM","SACE_YBN","SACE_YBO","SACE_YBP","SACE_YCJ","SACE_YCS","SACE_YCT","SACE_YCU","SACE_YCV","SACE_YCW",
             "SACE_YCX","SACE_YCY","SACE_YCZ","SACE_YDA","SACE_YDB","SACE_YDC","SACE_YDD", "SACE_YDE","SACE_YDF","SACE_YDG","SACE_YDH","SACE_YDI",
             "SACE_YDJ", "SACE_YDK", "SACE_YDL"]
print(len(to_delete))
index = 0
for index in range(len(to_delete)):
    curr = to_delete[index]
    curr = curr+"_"+curr
    to_delete[index] = curr
    index+=1
to_delete_index = []
for item in to_delete:
    curr_index = column_names.index(item)
    to_delete_index.append(curr_index)
snp_yeast_matrix = snp_yeast_matrix.drop(to_delete)
snp_yeast_matrix.head()

40


,0,1,2,3,4,5,6,7,8,9,...,18921,18922,18923,18924,18925,18926,18927,18928,18929,18930
AAA_AAA,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAB_AAB,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAC_AAC,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAD_AAD,0,0,0,0,1,1,1,0,0,0,...,0,0,1,1,1,1,0,0,1,1
AAE_AAE,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
pheno_url = "./phenoMatrix_35ConditionsNormalizedByYPD.csv"
pheno_yeast_matrix = pd.read_csv(pheno_url, delimiter='\t', error_bad_lines=False)
YPD6AU = pheno_yeast_matrix["YPD6AU"]
YPDBENOMYL200 = pheno_yeast_matrix["YPDBENOMYL200"]
YPDNACL1M = pheno_yeast_matrix["YPDNACL1M"]
YPDSDS = pheno_yeast_matrix["YPDSDS"]
YPGALACTOSE = pheno_yeast_matrix["YPGALACTOSE"]
YPGLYCEROL = pheno_yeast_matrix["YPGLYCEROL"]
YPRIBOSE = pheno_yeast_matrix["YPRIBOSE"]
YPSORBITOL = pheno_yeast_matrix["YPSORBITOL"]
total_run = 100

## Calculate index of top associated snps in 100 runs ##

In [4]:
sum = 0
weight_vector_1 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPD6AU, test_size=0.2)
    lr1 = linear_model.LinearRegression()
    lr1.fit(x_train, y_train)
    temp = np.array(lr1.coef_)
    weight_vector_1 = [x + y for x, y in zip(weight_vector_1, temp)]
    predictions_1 = lr1.predict(x_test)
    pcc_1 = stat.pearsonr(predictions_1, y_test)
    counter+=1
    sum += pcc_1[0]

avg_assoc_1 = sum/total_run
avg_weight_vector_1 = np.divide(weight_vector_1, total_run)
print(avg_assoc_1)
print(avg_weight_vector_1)
top2000_index = (-avg_weight_vector_1).argsort()[:2000]
print (top2000_index)
print (avg_weight_vector_1[top2000_index])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_2 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDBENOMYL200, test_size=0.2)
    lr2 = linear_model.LinearRegression()
    lr2.fit(x_train, y_train)
    temp = np.array(lr2.coef_)
    weight_vector_2 = [x + y for x, y in zip(weight_vector_2, temp)]
    predictions_2 = lr2.predict(x_test)
    pcc_2 = stat.pearsonr(predictions_2, y_test)
    counter+=1
    sum += pcc_2[0]

avg_assoc_2 = sum/total_run
avg_weight_vector_2 = np.divide(weight_vector_2, total_run)
print(avg_assoc_2)
print(avg_weight_vector_2)
top2000_index_2 = (-avg_weight_vector_2).argsort()[:2000]
print (top2000_index_2)
print (avg_weight_vector_2[top2000_index_2])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_3 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDNACL1M, test_size=0.2)
    lr3 = linear_model.LinearRegression()
    lr3.fit(x_train, y_train)
    temp = np.array(lr3.coef_)
    weight_vector_3 = [x + y for x, y in zip(weight_vector_3, temp)]
    predictions_3 = lr3.predict(x_test)
    pcc_3 = stat.pearsonr(predictions_3, y_test)
    counter+=1
    sum += pcc_3[0]

avg_assoc_3 = sum/total_run
avg_weight_vector_3 = np.divide(weight_vector_3, total_run)
print(avg_assoc_3)
print(avg_weight_vector_3)
top2000_index_3 = (-avg_weight_vector_3).argsort()[:2000]
print (top2000_index_3)
print (avg_weight_vector_3[top2000_index_3])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_4 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDSDS, test_size=0.2)
    lr4 = linear_model.LinearRegression()
    lr4.fit(x_train, y_train)
    temp = np.array(lr4.coef_)
    weight_vector_4 = [x + y for x, y in zip(weight_vector_4, temp)]
    predictions_4 = lr4.predict(x_test)
    pcc_4 = stat.pearsonr(predictions_4, y_test)
    counter+=1
    sum += pcc_4[0]

avg_assoc_4 = sum/total_run
avg_weight_vector_4 = np.divide(weight_vector_4, total_run)
print(avg_assoc_4)
print(avg_weight_vector_4)
top2000_index_4 = (-avg_weight_vector_4).argsort()[:2000]
print (top2000_index_4)
print (avg_weight_vector_4[top2000_index_4])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_5 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGALACTOSE, test_size=0.2)
    lr5 = linear_model.LinearRegression()
    lr5.fit(x_train, y_train)
    temp = np.array(lr5.coef_)
    weight_vector_5 = [x + y for x, y in zip(weight_vector_5, temp)]
    predictions_5 = lr5.predict(x_test)
    pcc_5 = stat.pearsonr(predictions_5, y_test)
    counter+=1
    sum += pcc_5[0]

avg_assoc_5 = sum/total_run
avg_weight_vector_5 = np.divide(weight_vector_5, total_run)
print(avg_assoc_5)
print(avg_weight_vector_5)
top2000_index_5 = (-avg_weight_vector_5).argsort()[:2000]
print (top2000_index_5)
print (avg_weight_vector_5[top2000_index_5])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_6 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGLYCEROL, test_size=0.2)
    lr6 = linear_model.LinearRegression()
    lr6.fit(x_train, y_train)
    temp = np.array(lr6.coef_)
    weight_vector_6 = [x + y for x, y in zip(weight_vector_6, temp)]
    predictions_6 = lr6.predict(x_test)
    pcc_6 = stat.pearsonr(predictions_6, y_test)
    counter+=1
    sum += pcc_6[0]

avg_assoc_6 = sum/total_run
avg_weight_vector_6 = np.divide(weight_vector_6, total_run)
print(avg_assoc_6)
print(avg_weight_vector_6)
top2000_index_6 = (-avg_weight_vector_6).argsort()[:2000]
print (top2000_index_6)
print (avg_weight_vector_6[top2000_index_6])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_7 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPRIBOSE, test_size=0.2)
    lr7 = linear_model.LinearRegression()
    lr7.fit(x_train, y_train)
    temp = np.array(lr7.coef_)
    weight_vector_7 = [x + y for x, y in zip(weight_vector_7, temp)]
    predictions_7 = lr7.predict(x_test)
    pcc_7 = stat.pearsonr(predictions_7, y_test)
    counter+=1
    sum += pcc_7[0]

avg_assoc_7 = sum/total_run
avg_weight_vector_7 = np.divide(weight_vector_7, total_run)
print(avg_assoc_7)
print(avg_weight_vector_7)
top2000_index_7 = (-avg_weight_vector_7).argsort()[:2000]
print (top2000_index_7)
print (avg_weight_vector_7[top2000_index_7])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_8 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPSORBITOL, test_size=0.2)
    lr8 = linear_model.LinearRegression()
    lr8.fit(x_train, y_train)
    temp = np.array(lr8.coef_)
    weight_vector_8 = [x + y for x, y in zip(weight_vector_8, temp)]
    predictions_8 = lr8.predict(x_test)
    pcc_8 = stat.pearsonr(predictions_8, y_test)
    counter+=1
    sum += pcc_8[0]

avg_assoc_8 = sum/total_run
avg_weight_vector_8 = np.divide(weight_vector_8, total_run)
print(avg_assoc_8)
print(avg_weight_vector_8)
top2000_index_8 = (-avg_weight_vector_8).argsort()[:2000]
print (top2000_index_8)
print (avg_weight_vector_8[top2000_index_8])

0.012257347815486095
[-4.70382797e+07 -6.89749386e+07 -1.83195975e+07 ...  2.27410609e+05
  1.05730682e+04 -2.23611876e+06]
[    3    21    10 ... 14519 14741  1267]
[63489555.34764801 37168502.67938235 25187533.21800183 ...
   966291.92206568   966192.06802127   966080.69411545]
0.02928149642082002
[11362998.05945773 28713208.33432954 13627430.90108617 ...
   -40442.2075441    165859.89389064   -39600.91799881]
[   1    3    7 ... 5896 1129 4482]
[28713208.33432954 23918107.46240122 22478385.36228473 ...
   322581.45664118   322579.76917957   322544.18034099]
0.03055584375431484
[-1.30255456e+07  1.16265475e+07  7.70332847e+07 ...  6.63462302e+04
  2.67797084e+05  1.03873146e+06]
[    2     5    11 ... 13456  9317 17605]
[77033284.7110372  34113975.27897751 28712000.58812311 ...
   985321.52233595   985242.63674778   984991.83188687]
0.02986436825179075
[-31589016.07814947 -32537335.13906072 -32012055.36644748 ...
    -50114.80156448   -819854.95537944   -350007.71854033]
[    4    37

In [5]:
valid_snp_list=[top2000_index,top2000_index_2,top2000_index_3,top2000_index_4,top2000_index_5,top2000_index_6,top2000_index_7,top2000_index_8]

In [108]:
def get_gene(input):
    words = input.split(';')
    gene_tuple = words[2]
    words = gene_tuple.split('=')
    gene = words[1]
    return gene

def snp_pos_to_gene(pos):
    index = bs.bisect(start_pos, pos)
    curr_start = start_pos[index-1]
    curr_tuple = dict[curr_start]
    curr_end = curr_tuple[2]
    if pos in range ((int)(curr_start), (int)(curr_end)):
        return curr_tuple[0]

names=["Contig", "Seq","Type", "Start", "End", "N/A", "N/A1", "N/A2", "Info"]
gene_matrix = pd.read_csv(Gene_url, sep = '\t', error_bad_lines=False, skiprows = 8, names = names)
gene_matrix.head()

dict = {}
gene_list = []
for row in gene_matrix.itertuples():
    if(row.Type == "gene"):
        curr_info = row.Info
        curr_gene = get_gene(curr_info)
        gene_list.append(curr_gene)
        curr_start = row.Start
        curr_end = row.End
        tuple = (curr_gene, curr_start, curr_end)
        dict[curr_start] = tuple
print("Dict finished. There are %u genes" %len(dict))
start_pos = list(dict.keys())
rows = snp_yeast_matrix.index.values
rows = rows[0:]
array = np.zeros([len(rows),len(gene_list)])

yeast_gene_matrix_1 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_2 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_3 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_4 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_5 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_6 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_7 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)
yeast_gene_matrix_8 = pd.DataFrame(array, columns = gene_list, index = rows, copy = True)

yeast_gene_matrix_list = [yeast_gene_matrix_1,yeast_gene_matrix_2,yeast_gene_matrix_3,yeast_gene_matrix_4,yeast_gene_matrix_5,yeast_gene_matrix_6,yeast_gene_matrix_7,yeast_gene_matrix_8]
yeast_gene_matrix_1.head()

Dict finished. There are 6376 genes


,PAU8,YAL067W-A,SEO1,YAL065C,YAL064W-B,TDA8,YAL064W,YAL063C-A,FLO9,GDH3,...,tN(GUU)Q,tM(CAU)Q1,COX2,Q0255,tF(GAA)Q,tT(UAG)Q2,tV(UAC)Q,COX3,tM(CAU)Q2,RPM1
AAA_AAA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAB_AAB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAC_AAC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAD_AAD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAE_AAE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
def snp_list_to_gene(valid_snp_list, snp_yeast_matrix, yeast_gene_matrix):
    max_snp = 18931
    max_yeast = 971
    snp_counter = 0
    for snp_counter in range(max_snp):
        if(snp_counter %2000 == 0):
            print(snp_counter)
        curr_column = snp_yeast_matrix[snp_counter].tolist()
        curr_pos = pos_list[snp_counter]
        yeast_counter = 0
        for yeast_counter in range (max_yeast):
            curr_val = curr_column[yeast_counter]
            if(curr_val)!=0:
                if snp_counter in valid_snp_list:
                    gene = snp_pos_to_gene(curr_pos)
                    if(gene is None):
                        continue
                    yeast_name = rows[yeast_counter]
                    yeast_gene_matrix[gene][yeast_name] += 1
            yeast_counter += 1
        snp_counter += 1
    print("done")


In [110]:
for valid_snp, yg_matrix in zip(valid_snp_list, yeast_gene_matrix_list):
    snp_list_to_gene(valid_snp, snp_yeast_matrix, yg_matrix)


0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done
0
2000
4000
6000
8000
10000
12000
14000
16000
18000
done


In [111]:
min_yeast_with_gene = 400
min_total_gene = 2000
for matrix in yeast_gene_matrix_list:
    for column in matrix:
        counter = 0
        value = 0
        temp = matrix[column]
        for element in temp:
            if element != 0:
                counter += 1
                value += 4
        if(counter < min_yeast_with_gene and value <min_total_gene ):
            del matrix[column]
            
for matrix in yeast_gene_matrix_list:
    print(matrix.shape)


(971, 251)
(971, 242)
(971, 259)
(971, 252)
(971, 251)
(971, 242)
(971, 235)
(971, 232)


In [151]:
yeast_gene_matrix_list[0]
 

,FLO9,BDH2,BDH1,CNE1,GPB2,ACS1,FLC2,OAF1,AIM2,GEM1,...,BET2,PRP4,HDA3,AOS1,GDB1,ATG13,PZF1,SKI3,RPC82,QCR2
AAA_AAA,1.0,0.0,1.0,0.0,1.0,3.0,2.0,1.0,1.0,1.0,...,3.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
AAB_AAB,1.0,0.0,1.0,1.0,2.0,3.0,1.0,0.0,0.0,0.0,...,0.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
AAC_AAC,4.0,1.0,1.0,1.0,1.0,2.0,2.0,4.0,1.0,2.0,...,0.0,2.0,2.0,1.0,0.0,2.0,0.0,1.0,3.0,1.0
AAD_AAD,8.0,1.0,4.0,6.0,6.0,8.0,5.0,12.0,3.0,2.0,...,0.0,2.0,2.0,2.0,5.0,5.0,0.0,3.0,4.0,1.0
AAE_AAE,1.0,1.0,1.0,1.0,2.0,2.0,0.0,1.0,0.0,1.0,...,2.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
AAG_AAG,10.0,0.0,3.0,3.0,2.0,6.0,4.0,6.0,3.0,0.0,...,0.0,3.0,1.0,2.0,2.0,4.0,0.0,2.0,0.0,1.0
AAH_AAH,8.0,1.0,2.0,4.0,5.0,6.0,3.0,6.0,3.0,0.0,...,2.0,2.0,2.0,1.0,3.0,3.0,0.0,2.0,2.0,1.0
AAI_AAI,1.0,1.0,0.0,0.0,1.0,4.0,2.0,1.0,1.0,0.0,...,3.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
AAK_AAK,1.0,1.0,0.0,1.0,2.0,1.0,0.0,1.0,0.0,1.0,...,2.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0
AAL_AAL,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,...,3.0,0.0,2.0,0.0,2.0,2.0,1.0,2.0,2.0,1.0


In [113]:
yeast_gene_matrix_list[1].head()


,FLO9,BDH2,BDH1,ECM1,CNE1,GPB2,ACS1,FLC2,OAF1,AIM2,...,TIF3,MMS1,BSP1,VPS4,DPB2,BET2,AOS1,GDB1,ATG13,PZF1
AAA_AAA,4.0,1.0,1.0,2.0,1.0,0.0,3.0,1.0,2.0,2.0,...,1.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0
AAB_AAB,0.0,1.0,0.0,1.0,3.0,1.0,3.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
AAC_AAC,6.0,0.0,0.0,2.0,2.0,0.0,2.0,2.0,1.0,0.0,...,1.0,2.0,1.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0
AAD_AAD,11.0,2.0,1.0,1.0,5.0,6.0,5.0,4.0,6.0,1.0,...,1.0,3.0,3.0,2.0,0.0,0.0,4.0,8.0,2.0,3.0
AAE_AAE,0.0,0.0,0.0,1.0,2.0,0.0,2.0,0.0,2.0,1.0,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


## Neural Network ##

In [129]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as dt

class Net(nn.Module):
    def __init__(self, input_size, h1, h2, h3, output_size, lr):
        super(Net, self).__init__()
        self.l1 = nn.Linear(input_size, h1)
        self.l2 = nn.Linear(h1, h2)
        self.l3 = nn.Linear(h2, h3)
        self.l4 = nn.Linear(h3, output_size)
        self.relu = nn.ReLU()
        
        self.loss_fn = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr)

    def forward(self,x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
        return out
     
    def train_net(self, num_epochs, train_loader, test_loader):
        for epoch in range (num_epochs):
            total_loss = 0
            counter = 0
            for i, (data, target) in enumerate(train_loader):
                self.zero_grad()
                data = data.to(device)
                target = target.to(device)
                output = self.forward(data).to(device)
                loss = self.loss_fn(output, target)
                total_loss += loss
                counter += 1
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
            check_correlation(test_loader, self)    
            epoch += 1
            print ('Epoch [{}/{}], Loss: {:.4f}' 
                .format(epoch, num_epochs, total_loss/len(train_loader)))
    
def check_correlation(test_loader, net):
    counter = 0
    with torch.no_grad():
        total_err = 0.0
        for (data, target) in test_loader:
            data = data.to(device)
            target = target.cpu().numpy()
            output = torch.squeeze(net(data)).cpu().numpy()
            result = stat.pearsonr(target, output)
    print("Correlation: %f \n P-Value: %f" %(result[0], result[1]))   

In [104]:
pheno_url = "./phenoMatrix_35ConditionsNormalizedByYPD.csv"
pheno_yeast_matrix = pd.read_csv(pheno_url, delimiter='\t', error_bad_lines=False)
YPD6AU = pheno_yeast_matrix["YPD6AU"]
YPDBENOMYL200 = pheno_yeast_matrix["YPDBENOMYL200"]
YPDNACL1M = pheno_yeast_matrix["YPDNACL1M"]
YPDSDS = pheno_yeast_matrix["YPDSDS"]
YPGALACTOSE = pheno_yeast_matrix["YPGALACTOSE"]
YPGLYCEROL = pheno_yeast_matrix["YPGLYCEROL"]
YPRIBOSE = pheno_yeast_matrix["YPRIBOSE"]
YPSORBITOL = pheno_yeast_matrix["YPSORBITOL"]

In [147]:
device = torch.device('cuda')
h1 = 100
h2 = 50
h3 = 20
output_size = 1
lr = 0.0001
num_epochs = 20
batch_size = 200
split = 0.8

In [143]:
training_data_0 = torch.tensor(yeast_gene_matrix_list[0].values, dtype = torch.float)
answer_0 = torch.tensor(YPD6AU, dtype = torch.float)
train_set_0 = dt.TensorDataset(training_data_0, answer_0)
train_size = int(split * len(train_set_0))
test_size = len(train_set_0) - train_size 
train_dataset_0, test_dataset_0 = dt.random_split(train_set_0, [train_size, test_size])
train_loader_0 = dt.DataLoader(dataset=train_dataset_0, batch_size=batch_size)
test_loader_0 = dt.DataLoader(dataset=test_dataset_0, batch_size=batch_size)


training_data_1 = torch.tensor(yeast_gene_matrix_list[1].values, dtype = torch.float)
answer_1 = torch.tensor(YPDBENOMYL200, dtype = torch.float)
train_set_1 = dt.TensorDataset(training_data_1, answer_1)
train_dataset_1, test_dataset_1 = dt.random_split(train_set_1, [train_size, test_size])
train_loader_1 = dt.DataLoader(dataset=train_dataset_1, batch_size=batch_size)
test_loader_1 = dt.DataLoader(dataset=test_dataset_1, batch_size=batch_size)


training_data_2 = torch.tensor(yeast_gene_matrix_list[2].values, dtype = torch.float)
answer_2 = torch.tensor(YPDNACL1M, dtype = torch.float)
train_set_2 = dt.TensorDataset(training_data_2, answer_2)
train_dataset_2, test_dataset_2 = dt.random_split(train_set_2, [train_size, test_size])
train_loader_2 = dt.DataLoader(dataset=train_dataset_2, batch_size=batch_size)
test_loader_2 = dt.DataLoader(dataset=test_dataset_2, batch_size=batch_size)


training_data_3 = torch.tensor(yeast_gene_matrix_list[3].values, dtype = torch.float)
answer_3 = torch.tensor(YPDSDS, dtype = torch.float)
train_set_3 = dt.TensorDataset(training_data_3, answer_3)
train_dataset_3, test_dataset_3 = dt.random_split(train_set_3, [train_size, test_size])
train_loader_3 = dt.DataLoader(dataset=train_dataset_3, batch_size=batch_size)
test_loader_3 = dt.DataLoader(dataset=test_dataset_3, batch_size=batch_size)

training_data_5 = torch.tensor(yeast_gene_matrix_list[4].values, dtype = torch.float)
answer_5 = torch.tensor(YPGALACTOSE, dtype = torch.float)
train_set_5 = dt.TensorDataset(training_data_5, answer_5)
train_dataset_5, test_dataset_5 = dt.random_split(train_set_5, [train_size, test_size])
train_loader_5 = dt.DataLoader(dataset=train_dataset_5, batch_size=batch_size)
test_loader_5 = dt.DataLoader(dataset=test_dataset_5, batch_size=batch_size)

training_data_6 = torch.tensor(yeast_gene_matrix_list[5].values, dtype = torch.float)
answer_6 = torch.tensor(YPGLYCEROL, dtype = torch.float)
train_set_6 = dt.TensorDataset(training_data_6, answer_6)
train_dataset_6, test_dataset_6 = dt.random_split(train_set_6, [train_size, test_size])
train_loader_6 = dt.DataLoader(dataset=train_dataset_6, batch_size=batch_size)
test_loader_6 = dt.DataLoader(dataset=test_dataset_6, batch_size=batch_size)

training_data_7 = torch.tensor(yeast_gene_matrix_list[6].values, dtype = torch.float)
answer_7 = torch.tensor(YPRIBOSE, dtype = torch.float)
train_set_7 = dt.TensorDataset(training_data_7, answer_7)
train_dataset_7, test_dataset_7 = dt.random_split(train_set_7, [train_size, test_size])
train_loader_7 = dt.DataLoader(dataset=train_dataset_7, batch_size=batch_size)
test_loader_7 = dt.DataLoader(dataset=test_dataset_7, batch_size=batch_size)

training_data_8 = torch.tensor(yeast_gene_matrix_list[7].values, dtype = torch.float)
answer_8 = torch.tensor(YPSORBITOL, dtype = torch.float)
train_set_8 = dt.TensorDataset(training_data_8, answer_8)
train_dataset_8, test_dataset_8 = dt.random_split(train_set_8, [train_size, test_size])
train_loader_8 = dt.DataLoader(dataset=train_dataset_8, batch_size=batch_size)
test_loader_8 = dt.DataLoader(dataset=test_dataset_8, batch_size=batch_size)



In [152]:
train_loader_list = [train_loader_0, train_loader_1, train_loader_2, train_loader_3, train_loader_5, train_loader_6, train_loader_7, train_loader_8]
test_loader_list = [test_loader_0, test_loader_1, test_loader_2, test_loader_3, test_loader_5, test_loader_6, test_loader_7, test_loader_8]

In [153]:
print(int(split * len(train_set_0)))

776


In [157]:
input_size = 251
net_0 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_0.train_net(num_epochs, train_loader_list[0], test_loader_list[0])

Correlation: 0.124832 
 P-Value: 0.082072
Epoch [1/20], Loss: 0.2319
Correlation: -0.067315 
 P-Value: 0.349776
Epoch [2/20], Loss: 0.1952
Correlation: -0.130510 
 P-Value: 0.068981
Epoch [3/20], Loss: 0.1680
Correlation: -0.158785 
 P-Value: 0.026613
Epoch [4/20], Loss: 0.1476
Correlation: -0.165046 
 P-Value: 0.021124
Epoch [5/20], Loss: 0.1310
Correlation: -0.167648 
 P-Value: 0.019148
Epoch [6/20], Loss: 0.1144
Correlation: -0.179667 
 P-Value: 0.011962
Epoch [7/20], Loss: 0.0990
Correlation: -0.190812 
 P-Value: 0.007540
Epoch [8/20], Loss: 0.0855
Correlation: -0.197603 
 P-Value: 0.005623
Epoch [9/20], Loss: 0.0747
Correlation: -0.199541 
 P-Value: 0.005163
Epoch [10/20], Loss: 0.0671
Correlation: -0.198234 
 P-Value: 0.005469
Epoch [11/20], Loss: 0.0624
Correlation: -0.194601 
 P-Value: 0.006409
Epoch [12/20], Loss: 0.0593
Correlation: -0.189995 
 P-Value: 0.007806
Epoch [13/20], Loss: 0.0567
Correlation: -0.182343 
 P-Value: 0.010731
Epoch [14/20], Loss: 0.0536
Correlation: -0.

In [160]:
input_size = 242
net_1 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_1.train_net(num_epochs, train_loader_list[1], test_loader_list[1])

Correlation: -0.078292 
 P-Value: 0.276623
Epoch [1/20], Loss: 0.1765
Correlation: 0.075574 
 P-Value: 0.293693
Epoch [2/20], Loss: 0.1357
Correlation: 0.117833 
 P-Value: 0.100882
Epoch [3/20], Loss: 0.1120
Correlation: 0.105876 
 P-Value: 0.140718
Epoch [4/20], Loss: 0.0983
Correlation: 0.101175 
 P-Value: 0.159320
Epoch [5/20], Loss: 0.0864
Correlation: 0.098158 
 P-Value: 0.172193
Epoch [6/20], Loss: 0.0748
Correlation: 0.103030 
 P-Value: 0.151772
Epoch [7/20], Loss: 0.0634
Correlation: 0.103226 
 P-Value: 0.150991
Epoch [8/20], Loss: 0.0526
Correlation: 0.108296 
 P-Value: 0.131816
Epoch [9/20], Loss: 0.0431
Correlation: 0.112613 
 P-Value: 0.117014
Epoch [10/20], Loss: 0.0361
Correlation: 0.114066 
 P-Value: 0.112334
Epoch [11/20], Loss: 0.0324
Correlation: 0.115281 
 P-Value: 0.108534
Epoch [12/20], Loss: 0.0309
Correlation: 0.114563 
 P-Value: 0.110766
Epoch [13/20], Loss: 0.0295
Correlation: 0.113011 
 P-Value: 0.115715
Epoch [14/20], Loss: 0.0274
Correlation: 0.108883 
 P-Va

In [162]:
input_size = 259
net_2 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_2.train_net(num_epochs, train_loader_list[2], test_loader_list[2])

Correlation: -0.158677 
 P-Value: 0.026718
Epoch [1/20], Loss: 0.0887
Correlation: -0.162855 
 P-Value: 0.022922
Epoch [2/20], Loss: 0.0676
Correlation: -0.164846 
 P-Value: 0.021283
Epoch [3/20], Loss: 0.0516
Correlation: -0.162861 
 P-Value: 0.022917
Epoch [4/20], Loss: 0.0404
Correlation: -0.157184 
 P-Value: 0.028199
Epoch [5/20], Loss: 0.0338
Correlation: -0.146429 
 P-Value: 0.041088
Epoch [6/20], Loss: 0.0296
Correlation: -0.135962 
 P-Value: 0.058065
Epoch [7/20], Loss: 0.0259
Correlation: -0.121257 
 P-Value: 0.091295
Epoch [8/20], Loss: 0.0229
Correlation: -0.101798 
 P-Value: 0.156752
Epoch [9/20], Loss: 0.0205
Correlation: -0.079758 
 P-Value: 0.267705
Epoch [10/20], Loss: 0.0188
Correlation: -0.057957 
 P-Value: 0.420938
Epoch [11/20], Loss: 0.0176
Correlation: -0.038341 
 P-Value: 0.594618
Epoch [12/20], Loss: 0.0165
Correlation: -0.022301 
 P-Value: 0.756982
Epoch [13/20], Loss: 0.0158
Correlation: -0.007561 
 P-Value: 0.916456
Epoch [14/20], Loss: 0.0153
Correlation: 0.

In [164]:
input_size = 252
net_3 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_3.train_net(num_epochs, train_loader_list[3], test_loader_list[3])

Correlation: -0.044662 
 P-Value: 0.535274
Epoch [1/20], Loss: 0.0425
Correlation: -0.056329 
 P-Value: 0.434122
Epoch [2/20], Loss: 0.0364
Correlation: -0.087976 
 P-Value: 0.221332
Epoch [3/20], Loss: 0.0340
Correlation: -0.134395 
 P-Value: 0.061045
Epoch [4/20], Loss: 0.0315
Correlation: -0.154572 
 P-Value: 0.030960
Epoch [5/20], Loss: 0.0306
Correlation: -0.159592 
 P-Value: 0.025843
Epoch [6/20], Loss: 0.0304
Correlation: -0.160977 
 P-Value: 0.024567
Epoch [7/20], Loss: 0.0303
Correlation: -0.163919 
 P-Value: 0.022033
Epoch [8/20], Loss: 0.0301
Correlation: -0.166243 
 P-Value: 0.020194
Epoch [9/20], Loss: 0.0299
Correlation: -0.164328 
 P-Value: 0.021699
Epoch [10/20], Loss: 0.0296
Correlation: -0.159321 
 P-Value: 0.026100
Epoch [11/20], Loss: 0.0294
Correlation: -0.156242 
 P-Value: 0.029169
Epoch [12/20], Loss: 0.0293
Correlation: -0.158975 
 P-Value: 0.026430
Epoch [13/20], Loss: 0.0292
Correlation: -0.163476 
 P-Value: 0.022399
Epoch [14/20], Loss: 0.0291
Correlation: -0

In [168]:
input_size = 251
net_4 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_4.train_net(num_epochs, train_loader_list[4], test_loader_list[4])

Correlation: 0.105213 
 P-Value: 0.143238
Epoch [1/20], Loss: 1.2636
Correlation: 0.088397 
 P-Value: 0.219123
Epoch [2/20], Loss: 1.0857
Correlation: 0.078971 
 P-Value: 0.272468
Epoch [3/20], Loss: 0.9330
Correlation: 0.075681 
 P-Value: 0.293010
Epoch [4/20], Loss: 0.7952
Correlation: 0.073721 
 P-Value: 0.305723
Epoch [5/20], Loss: 0.6726
Correlation: 0.073476 
 P-Value: 0.307339
Epoch [6/20], Loss: 0.5724
Correlation: 0.073139 
 P-Value: 0.309568
Epoch [7/20], Loss: 0.4965
Correlation: 0.072406 
 P-Value: 0.314460
Epoch [8/20], Loss: 0.4442
Correlation: 0.072024 
 P-Value: 0.317020
Epoch [9/20], Loss: 0.4126
Correlation: 0.071086 
 P-Value: 0.323382
Epoch [10/20], Loss: 0.3949
Correlation: 0.069901 
 P-Value: 0.331534
Epoch [11/20], Loss: 0.3807
Correlation: 0.068093 
 P-Value: 0.344225
Epoch [12/20], Loss: 0.3633
Correlation: 0.066243 
 P-Value: 0.357525
Epoch [13/20], Loss: 0.3432
Correlation: 0.064318 
 P-Value: 0.371694
Epoch [14/20], Loss: 0.3224
Correlation: 0.061211 
 P-Val

In [169]:
input_size = 242
net_5 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_5.train_net(num_epochs, train_loader_list[5], test_loader_list[5])

Correlation: -0.173120 
 P-Value: 0.015510
Epoch [1/20], Loss: 0.5555
Correlation: -0.115955 
 P-Value: 0.106469
Epoch [2/20], Loss: 0.4711
Correlation: 0.021205 
 P-Value: 0.768575
Epoch [3/20], Loss: 0.4161
Correlation: 0.094645 
 P-Value: 0.188140
Epoch [4/20], Loss: 0.3717
Correlation: 0.125924 
 P-Value: 0.079412
Epoch [5/20], Loss: 0.3286
Correlation: 0.144701 
 P-Value: 0.043561
Epoch [6/20], Loss: 0.2869
Correlation: 0.151643 
 P-Value: 0.034326
Epoch [7/20], Loss: 0.2477
Correlation: 0.155612 
 P-Value: 0.029835
Epoch [8/20], Loss: 0.2127
Correlation: 0.158460 
 P-Value: 0.026929
Epoch [9/20], Loss: 0.1834
Correlation: 0.161033 
 P-Value: 0.024516
Epoch [10/20], Loss: 0.1618
Correlation: 0.162963 
 P-Value: 0.022830
Epoch [11/20], Loss: 0.1482
Correlation: 0.164905 
 P-Value: 0.021236
Epoch [12/20], Loss: 0.1390
Correlation: 0.164384 
 P-Value: 0.021654
Epoch [13/20], Loss: 0.1310
Correlation: 0.163570 
 P-Value: 0.022322
Epoch [14/20], Loss: 0.1244
Correlation: 0.161649 
 P-V

In [174]:
input_size = 235
net_6 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_6.train_net(num_epochs, train_loader_list[6], test_loader_list[6])

Correlation: 0.041300 
 P-Value: 0.566467
Epoch [1/20], Loss: 0.1732
Correlation: 0.052337 
 P-Value: 0.467440
Epoch [2/20], Loss: 0.1323
Correlation: 0.054676 
 P-Value: 0.447755
Epoch [3/20], Loss: 0.1013
Correlation: 0.055344 
 P-Value: 0.442216
Epoch [4/20], Loss: 0.0806
Correlation: 0.055853 
 P-Value: 0.438024
Epoch [5/20], Loss: 0.0698
Correlation: 0.057385 
 P-Value: 0.425543
Epoch [6/20], Loss: 0.0648
Correlation: 0.057553 
 P-Value: 0.424185
Epoch [7/20], Loss: 0.0609
Correlation: 0.054532 
 P-Value: 0.448948
Epoch [8/20], Loss: 0.0560
Correlation: 0.050764 
 P-Value: 0.480951
Epoch [9/20], Loss: 0.0502
Correlation: 0.045973 
 P-Value: 0.523354
Epoch [10/20], Loss: 0.0448
Correlation: 0.040824 
 P-Value: 0.570956
Epoch [11/20], Loss: 0.0404
Correlation: 0.038163 
 P-Value: 0.596333
Epoch [12/20], Loss: 0.0369
Correlation: 0.037618 
 P-Value: 0.601587
Epoch [13/20], Loss: 0.0343
Correlation: 0.036183 
 P-Value: 0.615536
Epoch [14/20], Loss: 0.0324
Correlation: 0.029250 
 P-Val

In [177]:
input_size = 232
net_7 = Net(input_size, h1, h2, h3, output_size, lr).to(device)
net_7.train_net(num_epochs, train_loader_list[7], test_loader_list[7])

Correlation: 0.025626 
 P-Value: 0.722139
Epoch [1/20], Loss: 0.1032
Correlation: 0.215649 
 P-Value: 0.002464
Epoch [2/20], Loss: 0.0850
Correlation: 0.216961 
 P-Value: 0.002315
Epoch [3/20], Loss: 0.0719
Correlation: 0.219069 
 P-Value: 0.002091
Epoch [4/20], Loss: 0.0619
Correlation: 0.222339 
 P-Value: 0.001784
Epoch [5/20], Loss: 0.0536
Correlation: 0.225944 
 P-Value: 0.001493
Epoch [6/20], Loss: 0.0469
Correlation: 0.225601 
 P-Value: 0.001519
Epoch [7/20], Loss: 0.0426
Correlation: 0.220417 
 P-Value: 0.001959
Epoch [8/20], Loss: 0.0402
Correlation: 0.212082 
 P-Value: 0.002916
Epoch [9/20], Loss: 0.0373
Correlation: 0.200216 
 P-Value: 0.005010
Epoch [10/20], Loss: 0.0340
Correlation: 0.175210 
 P-Value: 0.014289
Epoch [11/20], Loss: 0.0315
Correlation: 0.145312 
 P-Value: 0.042674
Epoch [12/20], Loss: 0.0297
Correlation: 0.119611 
 P-Value: 0.095810
Epoch [13/20], Loss: 0.0283
Correlation: 0.092261 
 P-Value: 0.199560
Epoch [14/20], Loss: 0.0273
Correlation: 0.064299 
 P-Val